<a href="https://colab.research.google.com/github/ShovalBenjer/deep_learning_neural_networks/blob/main/Deep_exc_1_adir_shoval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
from torch import nn

class BTU(nn.Module):
    def __init__(self, T=0.01):
        super(BTU, self).__init__()
        self.T = T

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return 1 / (1 + torch.exp(-input / self.T))

class LogisticNeuralNetwork(nn.Module):
    def __init__(self, k, bypass=False):
        super(LogisticNeuralNetwork, self).__init__()
        self.k = k
        self.bypass = bypass
        self.hidden = nn.Linear(2, k)
        self.output = nn.Linear(k + (2 if bypass else 0), 1)
        self.activation = BTU()

    def weights_set(self, w, b, layer_name):
        layer = getattr(self, layer_name)
        if w.shape != layer.weight.data.shape or b.shape != layer.bias.data.shape:
            raise ValueError(f"Error: Dimensions do not match for {layer_name}")
        with torch.no_grad():
            layer.weight = nn.Parameter(w)
            layer.bias = nn.Parameter(b)

    def forward(self, x):
        h = self.activation(self.hidden(x))
        if self.bypass:
            h = torch.cat((h, x), dim=1)
        return self.activation(self.output(h))

def loss_fn(predicted, actual):
    return torch.sum((predicted - actual) ** 2) / predicted.shape[0]

In [16]:
x = torch.FloatTensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y = torch.FloatTensor([[0.], [1.], [1.], [0.]])

for k in [1, 2, 4]:
    print(f"\nk = {k}")
    bypass = k == 1
    model = LogisticNeuralNetwork(k, bypass)

    if k == 1:
        model.weights_set(torch.tensor([[20., -20.]]), torch.tensor([-10.]), 'hidden')
        model.weights_set(torch.tensor([[20., 20., 20.]]), torch.tensor([-30.]), 'output')
    elif k == 2:
        model.weights_set(torch.tensor([[20., -20.], [-20., 20.]]), torch.tensor([-15., -15.]), 'hidden')
        model.weights_set(torch.tensor([[20., 20.]]), torch.tensor([-25.]), 'output')
    elif k == 4:
        model.weights_set(torch.tensor([[-1., -1.], [-1., 1.], [1., -1.], [1., 1.]]), torch.tensor([0.5, -0.5, -0.5, -1.5]), 'hidden')
        model.weights_set(torch.tensor([[0., 1., 1., 0.]]), torch.tensor([-0.5]), 'output')

    print("Weights and biases:")
    for name, param in model.named_parameters():
        print(f"{name}:\n{param.data}")

    y_pred = model(x)
    loss_value = loss_fn(y_pred, y)
    print(f"Loss: {loss_value.item()}")

    print("Truth table:")
    for i in range(4):
        print(f"Input: {x[i].tolist()}, Predicted Output: {round(y_pred[i].item())}, Expected Output: {y[i].item()}")



k = 1
Weights and biases:
hidden.weight:
tensor([[ 20., -20.]])
hidden.bias:
tensor([-10.])
output.weight:
tensor([[20., 20., 20.]])
output.bias:
tensor([-30.])
Loss: 0.5
Truth table:
Input: [0.0, 0.0], Predicted Output: 0, Expected Output: 0.0
Input: [0.0, 1.0], Predicted Output: 0, Expected Output: 1.0
Input: [1.0, 0.0], Predicted Output: 1, Expected Output: 1.0
Input: [1.0, 1.0], Predicted Output: 1, Expected Output: 0.0

k = 2
Weights and biases:
hidden.weight:
tensor([[ 20., -20.],
        [-20.,  20.]])
hidden.bias:
tensor([-15., -15.])
output.weight:
tensor([[20., 20.]])
output.bias:
tensor([-25.])
Loss: 0.5
Truth table:
Input: [0.0, 0.0], Predicted Output: 0, Expected Output: 0.0
Input: [0.0, 1.0], Predicted Output: 0, Expected Output: 1.0
Input: [1.0, 0.0], Predicted Output: 0, Expected Output: 1.0
Input: [1.0, 1.0], Predicted Output: 0, Expected Output: 0.0

k = 4
Weights and biases:
hidden.weight:
tensor([[-1., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1.,  1.